# Inicializar la Dimensión Mensajero
Importaciones

In [3]:
import pandas as pd
from sqlalchemy import create_engine
import yaml

# Conexion a la base de datos

In [4]:
# Cargar configuración de conexión desde el archivo YAML
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

# Extraer Datos del Mensajero

In [5]:
# Query para extraer la información de la dimensión mensajero
query = """
SELECT
    cm.id AS mensajero_id,
    cm.user_id,
    cm.activo,
    cm.telefono,
    cm.url_foto,
    c.nombre AS ciudad_nombre,
    au.username,
    au.first_name,
    au.last_name,
    au.email
FROM
    public.clientes_mensajeroaquitoy cm
LEFT JOIN
    public.ciudad c ON cm.ciudad_operacion_id = c.ciudad_id
LEFT JOIN
    public.auth_user au ON cm.user_id = au.id;
"""

# Leer los datos desde la base de datos 'RAPIDOS-Y_FURIOSOS' a un DataFrame de Pandas
df_mensajero = pd.read_sql(query, ryf_conn)

# Mostrar los primeros registros para verificar la extracción
df_mensajero.head()

,mensajero_id,user_id,activo,telefono,url_foto,ciudad_nombre,username,first_name,last_name,email
0,8,327,True,310-300000,http:,PALMIRA,Luis Castro,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
1,10,328,True,310-300000,http:,CALI,Luis Valenciano,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
2,11,329,True,310-300000,http:,CALI,Carlos Valderrama,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
3,12,330,True,310-300000,http:,CALI,Faustino Asprilla,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
4,13,331,True,310-300000,http:,PASTO,GEOVANNY Hidalgo,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com


# Cargar la Dimensión Mensajero

In [6]:
# Cargar los datos a la base de datos 'ETL_RYF' como la dimensión 'dim_mensajero'
df_mensajero.to_sql('dim_mensajero', etl_conn, index=False, if_exists='replace')

50